In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torchvision.ops as ops
import random
import numpy as np

print(torch.__version__)

2.1.0+cu121


In [2]:
print(f"Is CUDA supported by this system?{torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")

Is CUDA supported by this system?True
CUDA version: 12.1


In [3]:
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    np.random.seed(seed)
    random.seed(seed)

In [4]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

In [5]:
train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

In [6]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [7]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, padding=2), #1x28x28 -> 32x28x28
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) #32x14x14
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding=2), #32x14x14 -> 64x14x14
            nn.ReLU(),
            nn.MaxPool2d(2, 2) #64x7x7
        )
        
        self.fcl = nn.Sequential(
            nn.Linear(in_features=64 * 7 * 7, out_features=1024),
            nn.Dropout(0.2),
            nn.Linear(in_features=1024, out_features=10)
        )
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.size(0), -1)
        out = self.fcl(x)
        return out

In [8]:
# Choose a loss function for training
loss_fn = nn.CrossEntropyLoss()
setup_seed(0)
device = torch.device("cuda")
model = MyModel().to(device)

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss, correct = 0, 0
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        
        train_loss += loss.item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    train_loss /= num_batches
    correct /= size
    print(f"Train Error: \n Accuracy: {(correct*100):>0.1f}%, Avg loss: {train_loss:>8f}")
    
    return train_loss, correct

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    
    return test_loss, correct

In [9]:
# Main function: train and test the model
def main(chosen=""):
    EPOCHS = 30
    if not bool(chosen):
        optimisers = ['SGD', 'Momentum', 'Adagrad', 'RMSprop', 'Adam']
    else:
        optimisers = [chosen]

    for myoptimizer in optimisers:
        # Create an instance of the model
        model = MyModel()
        print('Training with: '+myoptimizer+'...')
        if myoptimizer == 'SGD':
            optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
        elif myoptimizer == 'Momentum':
            optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
        elif myoptimizer == 'Adagrad':
            optimizer = torch.optim.Adagrad(model.parameters(), lr=1e-3)
        elif myoptimizer == 'RMSprop':
            optimizer = torch.optim.RMSprop(model.parameters(), lr=1e-3)
        elif myoptimizer == 'Adam':
            optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
        else:
            print("Wrong optimizer type")
            continue

    train_loss, test_loss = [], []
    train_acc, test_acc = [], []
    for t in range(EPOCHS):
        print(f"Epoch {t+1}\n-------------------------------")
        loss, acc = train_loop(train_dataloader, model, loss_fn, optimizer)
        train_loss.append(loss), train_acc.append(acc)

        loss, acc = test_loop(test_dataloader, model, loss_fn)
        test_loss.append(loss), test_acc.append(acc)
    print("Done!")
    
    return test_acc[-1], test_loss[-1]

In [10]:
layer = 2
acc_dict, fcl_dict, channels_dict, deform_dict = {},{},{},{}
test_acc, test_loss = main("Adam")
acc_dict[layer] = [(test_acc*100), test_loss]

Training with: Adam...
Epoch 1
-------------------------------
Train Error: 
 Accuracy: 85.3%, Avg loss: 0.408266
Test Error: 
 Accuracy: 88.8%, Avg loss: 0.313593 

Epoch 2
-------------------------------
Train Error: 
 Accuracy: 90.0%, Avg loss: 0.278080
Test Error: 
 Accuracy: 89.5%, Avg loss: 0.287211 

Epoch 3
-------------------------------
Train Error: 
 Accuracy: 91.2%, Avg loss: 0.243877
Test Error: 
 Accuracy: 89.7%, Avg loss: 0.284087 

Epoch 4
-------------------------------
Train Error: 
 Accuracy: 92.1%, Avg loss: 0.219319
Test Error: 
 Accuracy: 89.2%, Avg loss: 0.310676 

Epoch 5
-------------------------------
Train Error: 
 Accuracy: 92.8%, Avg loss: 0.198164
Test Error: 
 Accuracy: 89.7%, Avg loss: 0.285537 

Epoch 6
-------------------------------
Train Error: 
 Accuracy: 93.4%, Avg loss: 0.182924
Test Error: 
 Accuracy: 90.4%, Avg loss: 0.274848 

Epoch 7
-------------------------------
Train Error: 
 Accuracy: 93.8%, Avg loss: 0.168447
Test Error: 
 Accuracy: 90.4

In [11]:
acc_dict

{2: [89.32, 0.6893838126758102]}

In [12]:
class MyDynamicModel(nn.Module):
    def __init__(self, num_layers=2, num_fcl_layers=1, input_size=28):
        super(MyDynamicModel, self).__init__()
        self.num_layers = num_layers
        
        self.layers = nn.ModuleList()
        
        self.layers.append(nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, padding=2), #1x28x28 -> 32x28x28
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) #32x14x14
        ))
        input_size//=2
        for i in range(1, self.num_layers):
            self.layers.append(nn.Sequential(
            nn.Conv2d(in_channels=32*i, out_channels=32*(i+1), kernel_size=5, padding=2), #32x14x14 -> 64x14x14
            nn.ReLU(),
            nn.MaxPool2d(2, 2) #64x7x7
            ))
            input_size//=2
        
        self.fc_layers = nn.ModuleList()
        self.num_fcl_layers = num_fcl_layers
        
        output = 1024
        self.fc_layers.append(nn.Sequential(
                nn.Linear(in_features=64 * input_size * input_size, out_features=1024),
                nn.Dropout(0.2)
            ))
        
        prev_output = output
        output//=2
        for i in range(self.num_fcl_layers):
            self.fc_layers.append(nn.Sequential(
                nn.Linear(in_features=prev_output, out_features=output),
                nn.Dropout(0.2)
            ))
            prev_output = output
            output//=2
            
        nn.Linear(in_features=1024, out_features=10)
        
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)

        x = x.view(x.size(0), -1)
        
        for layer in self.fc_layers:
            x = layer(x)
        
        out = self.additional_fc(x)
        return out

In [13]:
for layer in range(3,5):
    print(f"No. of layers: {layer}")
    model = MyDynamicModel(num_layers=layer)
    test_acc, test_loss = main("Adam")
    acc_dict[layer] = [(test_acc*100), test_loss]
    print(acc_dict) 

No. of layers: 3
Training with: Adam...
Epoch 1
-------------------------------
Train Error: 
 Accuracy: 85.1%, Avg loss: 0.417351
Test Error: 
 Accuracy: 88.4%, Avg loss: 0.324096 

Epoch 2
-------------------------------
Train Error: 
 Accuracy: 89.9%, Avg loss: 0.281271
Test Error: 
 Accuracy: 88.6%, Avg loss: 0.303325 

Epoch 3
-------------------------------
Train Error: 
 Accuracy: 91.1%, Avg loss: 0.245987
Test Error: 
 Accuracy: 90.2%, Avg loss: 0.279941 

Epoch 4
-------------------------------
Train Error: 
 Accuracy: 92.0%, Avg loss: 0.220761
Test Error: 
 Accuracy: 90.2%, Avg loss: 0.279595 

Epoch 5
-------------------------------
Train Error: 
 Accuracy: 92.6%, Avg loss: 0.204487
Test Error: 
 Accuracy: 90.0%, Avg loss: 0.291337 

Epoch 6
-------------------------------
Train Error: 
 Accuracy: 93.2%, Avg loss: 0.186450
Test Error: 
 Accuracy: 90.3%, Avg loss: 0.283019 

Epoch 7
-------------------------------
Train Error: 
 Accuracy: 93.7%, Avg loss: 0.173789
Test Error:

Train Error: 
 Accuracy: 97.5%, Avg loss: 0.071406
Test Error: 
 Accuracy: 89.7%, Avg loss: 0.589405 

Epoch 28
-------------------------------
Train Error: 
 Accuracy: 97.3%, Avg loss: 0.075885
Test Error: 
 Accuracy: 90.2%, Avg loss: 0.591061 

Epoch 29
-------------------------------
Train Error: 
 Accuracy: 97.5%, Avg loss: 0.069378
Test Error: 
 Accuracy: 90.0%, Avg loss: 0.613006 

Epoch 30
-------------------------------
Train Error: 
 Accuracy: 97.5%, Avg loss: 0.071515
Test Error: 
 Accuracy: 89.5%, Avg loss: 0.610665 

Done!
{2: [89.32, 0.6893838126758102], 3: [89.32, 0.6365924760889096], 4: [89.49000000000001, 0.6106650411703025]}


In [14]:
for fcl_layer in range(2,7):
    print(f"No. of fcl: {fcl_layer}")
    model = MyDynamicModel(num_fcl_layers=fcl_layer)
    test_acc, test_loss = main("Adam")
    fcl_dict[fcl_layer] = [(test_acc*100), test_loss]
    print(fcl_dict) 

No. of fcl: 2
Training with: Adam...
Epoch 1
-------------------------------
Train Error: 
 Accuracy: 85.4%, Avg loss: 0.408927
Test Error: 
 Accuracy: 87.6%, Avg loss: 0.341121 

Epoch 2
-------------------------------
Train Error: 
 Accuracy: 89.7%, Avg loss: 0.281095
Test Error: 
 Accuracy: 90.0%, Avg loss: 0.291660 

Epoch 3
-------------------------------
Train Error: 
 Accuracy: 91.1%, Avg loss: 0.245244
Test Error: 
 Accuracy: 89.8%, Avg loss: 0.285025 

Epoch 4
-------------------------------
Train Error: 
 Accuracy: 92.0%, Avg loss: 0.220650
Test Error: 
 Accuracy: 90.3%, Avg loss: 0.279671 

Epoch 5
-------------------------------
Train Error: 
 Accuracy: 92.7%, Avg loss: 0.200565
Test Error: 
 Accuracy: 90.3%, Avg loss: 0.266852 

Epoch 6
-------------------------------
Train Error: 
 Accuracy: 93.0%, Avg loss: 0.187930
Test Error: 
 Accuracy: 90.0%, Avg loss: 0.293506 

Epoch 7
-------------------------------
Train Error: 
 Accuracy: 93.6%, Avg loss: 0.172682
Test Error: 
 

Train Error: 
 Accuracy: 97.8%, Avg loss: 0.061226
Test Error: 
 Accuracy: 89.6%, Avg loss: 0.678067 

Epoch 28
-------------------------------
Train Error: 
 Accuracy: 97.8%, Avg loss: 0.065097
Test Error: 
 Accuracy: 89.6%, Avg loss: 0.626646 

Epoch 29
-------------------------------
Train Error: 
 Accuracy: 97.8%, Avg loss: 0.064340
Test Error: 
 Accuracy: 89.9%, Avg loss: 0.645699 

Epoch 30
-------------------------------
Train Error: 
 Accuracy: 97.9%, Avg loss: 0.062561
Test Error: 
 Accuracy: 88.9%, Avg loss: 0.700886 

Done!
{2: [89.27000000000001, 0.6994404390833939], 3: [88.9, 0.7008858083919355]}
No. of fcl: 4
Training with: Adam...
Epoch 1
-------------------------------
Train Error: 
 Accuracy: 85.5%, Avg loss: 0.400507
Test Error: 
 Accuracy: 88.3%, Avg loss: 0.328898 

Epoch 2
-------------------------------
Train Error: 
 Accuracy: 90.0%, Avg loss: 0.277796
Test Error: 
 Accuracy: 89.1%, Avg loss: 0.295907 

Epoch 3
-------------------------------
Train Error: 
 Accur

Test Error: 
 Accuracy: 90.2%, Avg loss: 0.477408 

Epoch 23
-------------------------------
Train Error: 
 Accuracy: 97.3%, Avg loss: 0.075182
Test Error: 
 Accuracy: 88.9%, Avg loss: 0.541510 

Epoch 24
-------------------------------
Train Error: 
 Accuracy: 97.3%, Avg loss: 0.076094
Test Error: 
 Accuracy: 89.1%, Avg loss: 0.548389 

Epoch 25
-------------------------------
Train Error: 
 Accuracy: 97.4%, Avg loss: 0.073563
Test Error: 
 Accuracy: 89.8%, Avg loss: 0.563922 

Epoch 26
-------------------------------
Train Error: 
 Accuracy: 97.6%, Avg loss: 0.068267
Test Error: 
 Accuracy: 89.5%, Avg loss: 0.568937 

Epoch 27
-------------------------------
Train Error: 
 Accuracy: 97.6%, Avg loss: 0.069637
Test Error: 
 Accuracy: 89.0%, Avg loss: 0.627202 

Epoch 28
-------------------------------
Train Error: 
 Accuracy: 97.4%, Avg loss: 0.072959
Test Error: 
 Accuracy: 89.8%, Avg loss: 0.645429 

Epoch 29
-------------------------------
Train Error: 
 Accuracy: 97.8%, Avg loss: 0

In [15]:
class MyDynamicModel2(nn.Module):
    def __init__(self, num_channels=8):
        super(MyDynamicModel2, self).__init__()
        self.num_channels = num_channels
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=self.num_channels//2, kernel_size=5, padding=2), #1x28x28 -> num_channelsx28x28
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) #num_channelsx14x14
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=self.num_channels//2, out_channels=self.num_channels, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(2, 2) #num_channelsx7x7
        )

        self.fcl = nn.Sequential(
            nn.Linear(in_features=self.num_channels * 7 * 7, out_features=1024),
            nn.Dropout(0.2),
            nn.Linear(in_features=1024, out_features=10)
        )
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.size(0), -1)
        out = self.fcl(x)
        return out

In [16]:
for num_channels in range(3,8):
    print(f"No. of channels: {2**num_channels}")
    model = MyDynamicModel2(num_channels=2**num_channels)
    test_acc, test_loss = main("Adam")
    channels_dict[num_channels] = [(test_acc*100), test_loss] 
    print(channels_dict) 

No. of channels: 8
Training with: Adam...
Epoch 1
-------------------------------
Train Error: 
 Accuracy: 84.8%, Avg loss: 0.421276
Test Error: 
 Accuracy: 88.5%, Avg loss: 0.318389 

Epoch 2
-------------------------------
Train Error: 
 Accuracy: 89.6%, Avg loss: 0.289018
Test Error: 
 Accuracy: 89.0%, Avg loss: 0.302600 

Epoch 3
-------------------------------
Train Error: 
 Accuracy: 91.1%, Avg loss: 0.249632
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.295702 

Epoch 4
-------------------------------
Train Error: 
 Accuracy: 91.8%, Avg loss: 0.228284
Test Error: 
 Accuracy: 90.0%, Avg loss: 0.279947 

Epoch 5
-------------------------------
Train Error: 
 Accuracy: 92.6%, Avg loss: 0.204420
Test Error: 
 Accuracy: 89.6%, Avg loss: 0.294140 

Epoch 6
-------------------------------
Train Error: 
 Accuracy: 93.0%, Avg loss: 0.193064
Test Error: 
 Accuracy: 89.8%, Avg loss: 0.293937 

Epoch 7
-------------------------------
Train Error: 
 Accuracy: 93.5%, Avg loss: 0.175463
Test Erro

Train Error: 
 Accuracy: 97.5%, Avg loss: 0.071296
Test Error: 
 Accuracy: 89.6%, Avg loss: 0.589606 

Epoch 28
-------------------------------
Train Error: 
 Accuracy: 97.6%, Avg loss: 0.067692
Test Error: 
 Accuracy: 90.0%, Avg loss: 0.625819 

Epoch 29
-------------------------------
Train Error: 
 Accuracy: 97.8%, Avg loss: 0.065071
Test Error: 
 Accuracy: 89.5%, Avg loss: 0.640559 

Epoch 30
-------------------------------
Train Error: 
 Accuracy: 97.6%, Avg loss: 0.068494
Test Error: 
 Accuracy: 89.7%, Avg loss: 0.672959 

Done!
{3: [89.13, 0.6842881385592898], 4: [89.71000000000001, 0.6729590664380438]}
No. of channels: 32
Training with: Adam...
Epoch 1
-------------------------------
Train Error: 
 Accuracy: 84.2%, Avg loss: 0.444853
Test Error: 
 Accuracy: 87.3%, Avg loss: 0.367609 

Epoch 2
-------------------------------
Train Error: 
 Accuracy: 89.3%, Avg loss: 0.297931
Test Error: 
 Accuracy: 88.3%, Avg loss: 0.312423 

Epoch 3
-------------------------------
Train Error: 

Test Error: 
 Accuracy: 89.7%, Avg loss: 0.452219 

Epoch 23
-------------------------------
Train Error: 
 Accuracy: 96.5%, Avg loss: 0.095557
Test Error: 
 Accuracy: 89.5%, Avg loss: 0.469272 

Epoch 24
-------------------------------
Train Error: 
 Accuracy: 96.6%, Avg loss: 0.093517
Test Error: 
 Accuracy: 89.9%, Avg loss: 0.447385 

Epoch 25
-------------------------------
Train Error: 
 Accuracy: 96.7%, Avg loss: 0.090451
Test Error: 
 Accuracy: 89.3%, Avg loss: 0.508241 

Epoch 26
-------------------------------
Train Error: 
 Accuracy: 96.9%, Avg loss: 0.088348
Test Error: 
 Accuracy: 89.5%, Avg loss: 0.533469 

Epoch 27
-------------------------------
Train Error: 
 Accuracy: 96.8%, Avg loss: 0.087645
Test Error: 
 Accuracy: 89.1%, Avg loss: 0.532142 

Epoch 28
-------------------------------
Train Error: 
 Accuracy: 97.1%, Avg loss: 0.082068
Test Error: 
 Accuracy: 88.4%, Avg loss: 0.541410 

Epoch 29
-------------------------------
Train Error: 
 Accuracy: 97.1%, Avg loss: 0

In [17]:
class DeformCNN(nn.Module):

    def __init__(self):
        super(DeformCNN, self).__init__()

        self.layer = ops.DeformConv2d(in_channels=1, out_channels=64, kernel_size=3, stride=1, padding=1, dilation=1)
        kernel_height, kernel_width = self.layer.kernel_size
        offset_channels = 2 * kernel_height * kernel_width
        self.offsets = nn.Conv2d(1, offset_channels, kernel_size=3, padding=1)
        self.out_channels = 32

        self.layer2 = ops.DeformConv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1, dilation=1)
        kernel_height2, kernel_width2 = self.layer2.kernel_size
        offset_channels2 = 2 * kernel_height2 * kernel_width2
        self.offsets2 = nn.Conv2d(64, offset_channels2, kernel_size=3, padding=1)
        self.out_channels = 64

        self.fc1 = nn.Linear(in_features=64*28*28, out_features=600)
        self.drop = nn.Dropout2d(0.25)
        self.fc2 = nn.Linear(in_features=600, out_features=120)
        self.fc3 = nn.Linear(in_features=120, out_features=10)

    def forward(self, x):
        offset = self.offsets(x)
        # print('offset1: ', offset.shape)
        out = self.layer(x.float(), offset.float()).to(x)
        # print('out1: ', out.shape)
        # print(out.shape)
        offset2 = self.offsets2(out)
        # print('offset2: ', offset2.shape)
        out = self.layer2(out.float(), offset2.float()).to(out)
        # print('out2: ', out)

        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.fc3(out)

        return out

In [19]:
model = DeformCNN()
test_acc, test_loss = main("Adam")
deform_dict = [(test_acc*100), test_loss]
print(deform_dict)

Training with: Adam...
Epoch 1
-------------------------------
Train Error: 
 Accuracy: 85.7%, Avg loss: 0.391619
Test Error: 
 Accuracy: 88.3%, Avg loss: 0.314148 

Epoch 2
-------------------------------
Train Error: 
 Accuracy: 90.2%, Avg loss: 0.272941
Test Error: 
 Accuracy: 89.6%, Avg loss: 0.277779 

Epoch 3
-------------------------------
Train Error: 
 Accuracy: 91.5%, Avg loss: 0.238613
Test Error: 
 Accuracy: 89.8%, Avg loss: 0.282961 

Epoch 4
-------------------------------
Train Error: 
 Accuracy: 92.2%, Avg loss: 0.213255
Test Error: 
 Accuracy: 90.0%, Avg loss: 0.289025 

Epoch 5
-------------------------------
Train Error: 
 Accuracy: 93.0%, Avg loss: 0.193641
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.290033 

Epoch 6
-------------------------------
Train Error: 
 Accuracy: 93.5%, Avg loss: 0.178123
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.287284 

Epoch 7
-------------------------------
Train Error: 
 Accuracy: 93.9%, Avg loss: 0.165359
Test Error: 
 Accuracy: 90.4

In [ ]:
[89.53999999999999, 0.633607772801807]
